<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Testing" data-toc-modified-id="Testing-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Testing</a></span><ul class="toc-item"><li><span><a href="#The-go-test-Tool" data-toc-modified-id="The-go-test-Tool-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>The go test Tool</a></span></li><li><span><a href="#Test-Functions" data-toc-modified-id="Test-Functions-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Test Functions</a></span><ul class="toc-item"><li><span><a href="#Randomized-Testing" data-toc-modified-id="Randomized-Testing-11.2.1"><span class="toc-item-num">11.2.1&nbsp;&nbsp;</span>Randomized Testing</a></span></li><li><span><a href="#Testing-a-Command" data-toc-modified-id="Testing-a-Command-11.2.2"><span class="toc-item-num">11.2.2&nbsp;&nbsp;</span>Testing a Command</a></span></li><li><span><a href="#White-Box-Testing" data-toc-modified-id="White-Box-Testing-11.2.3"><span class="toc-item-num">11.2.3&nbsp;&nbsp;</span>White-Box Testing</a></span></li></ul></li><li><span><a href="#Coverage" data-toc-modified-id="Coverage-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Coverage</a></span></li><li><span><a href="#Benchmark-Functions" data-toc-modified-id="Benchmark-Functions-11.4"><span class="toc-item-num">11.4&nbsp;&nbsp;</span>Benchmark Functions</a></span></li><li><span><a href="#Profiling" data-toc-modified-id="Profiling-11.5"><span class="toc-item-num">11.5&nbsp;&nbsp;</span>Profiling</a></span></li><li><span><a href="#Example-Functions" data-toc-modified-id="Example-Functions-11.6"><span class="toc-item-num">11.6&nbsp;&nbsp;</span>Example Functions</a></span></li></ul></li></ul></div>

# Testing

## The go test Tool

## Test Functions

In [8]:
import "unicode"
func IsPalindrome(s string) bool {
	var letters []rune
	for _, r := range s {
		if unicode.IsLetter(r) {
			letters = append(letters, unicode.ToLower(r))
		}
	}
	for i := range letters {
		if letters[i] != letters[len(letters)-1-i] {
			return false
		}
	}
	return true
}

### Randomized Testing

In [3]:
import "math/rand"
func randomPalindrome(rng *rand.Rand) string {
	n := rng.Intn(25) // random length up to 24
	runes := make([]rune, n)
	for i := 0; i < (n+1)/2; i++ {
		r := rune(rng.Intn(0x1000)) // random rune up to '\u0999'
		runes[i] = r
		runes[n-1-i] = r
	}
	return string(runes)
}

In [10]:
import "testing"
import "time"
func TestRandomPalindromes(t *testing.T) {
	// Initialize a pseudo-random number generator.
	seed := time.Now().UTC().UnixNano()
	t.Logf("Random seed: %d", seed)
	rng := rand.New(rand.NewSource(seed))

	for i := 0; i < 1000; i++ {
		p := randomPalindrome(rng)
		if !IsPalindrome(p) {
			t.Errorf("IsPalindrome(%q) = false", p)
		}
	}
}

### Testing a Command

In [14]:
//gopl.io/ch11/echo
import (
	"flag"
	"fmt"
	"io"
	"os"
	"strings"
)

var (
	n = flag.Bool("n", false, "omit trailing newline")
	s = flag.String("s", " ", "separator")
)
var out io.Writer = os.Stdout // modified during testing

func main() {
	flag.Parse()
	if err := echo(!*n, *s, flag.Args()); err != nil {
		fmt.Fprintf(os.Stderr, "echo: %v\n", err)
		os.Exit(1)
	}
}

func echo(newline bool, sep string, args []string) error {
	fmt.Fprint(out, strings.Join(args, sep))
	if newline {
		fmt.Fprintln(out)
	}
	return nil
}

gophernotes flag redefined: n


ERROR: gophernotes flag redefined: n

In [17]:
// Test of echo command.  Run with: go test gopl.io/ch11/echo

//!+
//package main

import (
	"bytes"
	"fmt"
	"testing"
)

func TestEcho(t *testing.T) {
	var tests = []struct {
		newline bool
		sep     string
		args    []string
		want    string
	}{
		{true, "", []string{}, "\n"},
		{false, "", []string{}, ""},
		{true, "\t", []string{"one", "two", "three"}, "one\ttwo\tthree\n"},
		{true, ",", []string{"a", "b", "c"}, "a,b,c\n"},
		{false, ":", []string{"1", "2", "3"}, "1:2:3"},
	}

	for _, test := range tests {
		descr := fmt.Sprintf("echo(%v, %q, %q)",
			test.newline, test.sep, test.args)

		out = new(bytes.Buffer) // captured output
		if err := echo(test.newline, test.sep, test.args); err != nil {
			t.Errorf("%s failed: %v", descr, err)
			continue
		}
		got := out.(*bytes.Buffer).String()
		if got != test.want {
			t.Errorf("%s = %q, want %q", descr, got, test.want)
		}
	}
}


### White-Box Testing

In [20]:
//gopl.io/ch11/storage1
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 311.

// Package storage is part of a hypothetical cloud storage server.
//!+main
//package storage

import (
	"fmt"
	"log"
	"net/smtp"
)

var usage = make(map[string]int64)

func bytesInUse(username string) int64 { return usage[username] }

// Email sender configuration.
// NOTE: never put passwords in source code!
const sender = "notifications@example.com"
const password = "correcthorsebatterystaple"
const hostname = "smtp.example.com"

const template = `Warning: you are using %d bytes of storage,
%d%% of your quota.`

func CheckQuota(username string) {
	used := bytesInUse(username)
	const quota = 1000000000 // 1GB
	percent := 100 * used / quota
	if percent < 90 {
		return // OK
	}
	msg := fmt.Sprintf(template, used, percent)
	auth := smtp.PlainAuth("", sender, password, hostname)
	err := smtp.SendMail(hostname+":587", auth, sender,
		[]string{username}, []byte(msg))
	if err != nil {
		log.Printf("smtp.SendMail(%s) failed: %s", username, err)
	}
}

//!-main

In [28]:
//gopl.io/ch11/storage2
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

// See page 312.

// Package storage is part of a hypothetical cloud storage server.
//package storage

import (
	"fmt"
	"log"
	"net/smtp"
)

var usage = make(map[string]int64)

func bytesInUse(username string) int64 { return usage[username] }

// E-mail sender configuration.
// NOTE: never put passwords in source code!
const sender = "notifications@example.com"
const password = "correcthorsebatterystaple"
const hostname = "smtp.example.com"

const template = `Warning: you are using %d bytes of storage,
%d%% of your quota.`

//!+factored
var notifyUser = func(username, msg string) {
	auth := smtp.PlainAuth("", sender, password, hostname)
	err := smtp.SendMail(hostname+":587", auth, sender,
		[]string{username}, []byte(msg))
	if err != nil {
		log.Printf("smtp.SendMail(%s) failed: %s", username, err)
	}
}

func CheckQuota(username string) {
	used := bytesInUse(username)
	const quota = 1000000000 // 1GB
	percent := 100 * used / quota
	if percent < 90 {
		return // OK
	}
	msg := fmt.Sprintf(template, used, percent)
	notifyUser(username, msg)
}

//!-factored

In [29]:
//gopl.io/ch11/quota_test.go
// Copyright © 2016 Alan A. A. Donovan & Brian W. Kernighan.
// License: https://creativecommons.org/licenses/by-nc-sa/4.0/

//!+test
//package storage

import (
	"strings"
	"testing"
)

func TestCheckQuotaNotifiesUser(t *testing.T) {
	var notifiedUser, notifiedMsg string
	notifyUser = func(user, msg string) {
		notifiedUser, notifiedMsg = user, msg
	}

	const user = "joe@example.org"
	usage[user] = 980000000 // simulate a 980MB-used condition

	CheckQuota(user)
	if notifiedUser == "" && notifiedMsg == "" {
		t.Fatalf("notifyUser not called")
	}
	if notifiedUser != user {
		t.Errorf("wrong user (%s) notified, want %s",
			notifiedUser, user)
	}
	const wantSubstring = "98% of your quota"
	if !strings.Contains(notifiedMsg, wantSubstring) {
		t.Errorf("unexpected notification message <<%s>>, "+
			"want substring %q", notifiedMsg, wantSubstring)
	}
}

//!-test

/*
//!+defer
func TestCheckQuotaNotifiesUser(t *testing.T) {
	// Save and restore original notifyUser.
	saved := notifyUser
	defer func() { notifyUser = saved }()
	// Install the test's fake notifyUser.
	var notifiedUser, notifiedMsg string
	notifyUser = func(user, msg string) {
		notifiedUser, notifiedMsg = user, msg
	}
	// ...rest of test...
}
//!-defer
*/

## Coverage

[coverage.html](cover120216535/coverage.html)

## Benchmark Functions

## Profiling

## Example Functions